# QAOA Parametrization Comparison: House Graph MaxCut

This notebook compares three QAOA parametrization strategies on the 5-node 'house graph':

1. **Vanilla QAOA**: Standard parametrization with 2p parameters
2. **Free QAOA**: One parameter per edge per layer (multi-angle)
3. **Orbit QAOA**: Parameters grouped by graph automorphism orbits

Based on: https://arxiv.org/pdf/2410.05187

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

from qaoa import QAOA, problems, mixers, initialstates
from qaoa.util import print_orbit_structure
from qiskit_algorithms.optimizers import COBYLA

## 1. Create the House Graph

The house graph has 5 nodes and 6 edges with clear symmetry structure.

In [ ]:
# Create house graph: square base with triangular roof
G_house = nx.Graph()
G_house.add_edges_from([
    (0, 1),  # Base square
    (1, 2),
    (2, 3),
    (3, 0),
    (0, 4),  # Roof triangle
    (1, 4)
])

# Draw graph
pos = {0: (0, 0), 1: (1, 0), 2: (1, 1), 3: (0, 1), 4: (0.5, 1.5)}
nx.draw(G_house, pos, with_labels=True, node_color='lightblue',
        node_size=500, font_size=16)
plt.title("House Graph")
plt.show()

print(f"Nodes: {G_house.number_of_nodes()}")
print(f"Edges: {G_house.number_of_edges()}")
print(f"Edges: {list(G_house.edges())}")

## 2. Compute Orbit Structure

Analyze the graph's symmetries to identify edge orbits.

In [ ]:
print_orbit_structure(G_house)

## 3. Compute Optimal MaxCut

Brute force search for the optimal solution.

In [ ]:
def compute_optimal_maxcut(G):
    """Compute optimal MaxCut value by brute force."""
    n = G.number_of_nodes()
    max_cut = 0
    best_partition = None

    for i in range(2**n):
        partition = format(i, f'0{n}b')
        cut = sum(1 for u, v in G.edges() if partition[u] != partition[v])
        if cut > max_cut:
            max_cut = cut
            best_partition = partition

    return max_cut, best_partition

optimal_cut, optimal_partition = compute_optimal_maxcut(G_house)
print(f"Optimal MaxCut value: {optimal_cut}")
print(f"Optimal partition: {optimal_partition}")

## 4. Run QAOA Comparisons

Compare three parametrization strategies at various depths.

In [ ]:
depths = [1, 2, 3, 4, 5]
results = {
    'Vanilla': {'ratios': [], 'params': [], 'costs': []},
    'Orbit': {'ratios': [], 'params': [], 'costs': []}
}

np.random.seed(42)  # For reproducibility

# Vanilla QAOA uses GraphProblem (MaxCut with standard 2p parameters)
vanilla_problem = problems.MaxKCutBinaryPowerOfTwo(G_house, k_cuts=2)

# Orbit QAOA uses MaxCutOrbit (one parameter per orbit per layer)
orbit_problem = problems.MaxCutOrbit(G_house)

for depth in depths:
    print(f"\n=== Depth {depth} ===")

    # 1. Vanilla QAOA (2*depth parameters: depth betas + depth gammas)
    print("Running Vanilla QAOA...")
    qaoa_vanilla = QAOA(
        vanilla_problem,
        mixers.X(),
        initialstates.Plus(),
        optimizer=[COBYLA, {'maxiter': 1000}]
    )
    qaoa_vanilla.optimize(depth=depth, initial_guess=np.random.rand(2 * depth))
    best_cost_vanilla = -qaoa_vanilla.get_Exp(depth)
    ratio_vanilla = best_cost_vanilla / optimal_cut
    results['Vanilla']['ratios'].append(ratio_vanilla)
    results['Vanilla']['params'].append(2 * depth)
    results['Vanilla']['costs'].append(best_cost_vanilla)
    print(f"  Vanilla: {2*depth} params, ratio={ratio_vanilla:.4f}")

    # 2. Orbit QAOA (depth * num_orbits parameters)
    print("Running Orbit QAOA...")
    qaoa_orbit = QAOA(
        orbit_problem,
        mixers.X(),
        initialstates.Plus(),
        optimizer=[COBYLA, {'maxiter': 1000}]
    )
    num_params = depth * orbit_problem.num_orbits
    qaoa_orbit.optimize(depth=depth, initial_guess=np.random.rand(num_params))
    best_cost_orbit = -qaoa_orbit.get_Exp(depth)
    ratio_orbit = best_cost_orbit / optimal_cut
    results['Orbit']['ratios'].append(ratio_orbit)
    results['Orbit']['params'].append(num_params)
    results['Orbit']['costs'].append(best_cost_orbit)
    print(f"  Orbit: {num_params} params, ratio={ratio_orbit:.4f}")

## 5. Visualize Results

Plot approximation ratios vs. circuit depth and number of parameters.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

colors = {'Vanilla': 'blue', 'Orbit': 'green'}

# Plot 1: Approximation Ratio vs Depth
for strategy in ['Vanilla', 'Orbit']:
    if results[strategy]['ratios']:
        ax1.plot(
            depths[:len(results[strategy]['ratios'])],
            results[strategy]['ratios'],
            marker='o', label=strategy, linewidth=2, color=colors[strategy]
        )

ax1.axhline(y=1.0, color='gray', linestyle='--', label='Optimal')
ax1.set_xlabel('Circuit Depth (p)', fontsize=12)
ax1.set_ylabel('Approximation Ratio', fontsize=12)
ax1.set_title('QAOA Performance vs Depth', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Approximation Ratio vs Number of Parameters
for strategy in ['Vanilla', 'Orbit']:
    if results[strategy]['ratios']:
        ax2.plot(
            results[strategy]['params'],
            results[strategy]['ratios'],
            marker='o', label=strategy, linewidth=2, color=colors[strategy]
        )

ax2.axhline(y=1.0, color='gray', linestyle='--', label='Optimal')
ax2.set_xlabel('Number of Parameters', fontsize=12)
ax2.set_ylabel('Approximation Ratio', fontsize=12)
ax2.set_title('QAOA Performance vs Parameter Count', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Summary Table

In [ ]:
import pandas as pd

summary_data = []
for strategy in ['Vanilla', 'Orbit']:
    if results[strategy]['ratios']:
        for i, depth in enumerate(depths[:len(results[strategy]['ratios'])]):
            summary_data.append({
                'Strategy': strategy,
                'Depth': depth,
                'Parameters': results[strategy]['params'][i],
                'Best Cut': f"{results[strategy]['costs'][i]:.4f}",
                'Approximation Ratio': f"{results[strategy]['ratios'][i]:.4f}"
            })

df = pd.DataFrame(summary_data)
print(df.to_string(index=False))

## Conclusions

**Expected Results (based on https://arxiv.org/pdf/2410.05187):**

1. **Vanilla QAOA**: Uses 2p parameters (1 beta + 1 gamma per layer). Simple but uniform across all edges.
2. **Orbit QAOA**: Uses p × |Orbits| parameters. Balances performance and efficiency by exploiting graph symmetries.

The orbit-based approach exploits graph symmetries to reduce parameters while maintaining
competitive approximation ratios. For the house graph with 6 edges and 4 orbits, orbit QAOA
uses more parameters per layer than vanilla (4 vs 1 gamma), but groups symmetrically
equivalent edges to preserve physical insight.

**Note:** Free QAOA (one parameter per edge) requires a multi-angle implementation and
will be added in a future update.